# nn.Embedding
`nn.Embedding` 层可以被理解为一个简单的查找表 (lookup table)，它储存了对应于一个固定词典大小的、固定维度的向量。它最核心的应用场景是将代表单词的整数索引转换为密集的、可学习的向量（即“词向量”）。

### 核心功能：什么是 `nn.Embedding`？ 📚
想象一下，你有一个巨大的表格（或数组），表格的每一行都是一个向量。`nn.Embedding` 的工作方式就是：你给它一个整数索引（行号），它就从表格中把对应行的向量取出来给你。

- 这个过程的关键在于，表格中存储的所有向量都是可学习的参数。在神经网络的训练过程中，模型会通过反向传播算法来不断微调这些向量，目的是让它们能更好地表达词与词之间的关系。例如，经过充分训练后，“国王”和“女王”的词向量在向量空间中的位置可能会非常接近。

---
### 为什么需要 `nn.Embedding`？ 🤔
它的主要目的是处理`类别型数据` (categorical data)，尤其是在`自然语言处理` (NLP) 领域中处理单词。计算机本身无法理解“猫”或“狗”这样的文本，我们必须先把它们转换成数值形式。

- 一种简单的方法是独热编码 (`One-Hot Encoding`)。例如，在一个很小的词汇表中，“猫”可能表示为 [1, 0, 0, 0]，“狗”表示为 [0, 1, 0, 0]。但这种方法有两个致命的缺点：

    - 维度灾难与稀疏性：如果你的词汇表里有10,000个单词，那么每个单词的向量都是10,000维的，其中只有一个元素是1，其他全是0。这在计算上非常低效，并且造成了数据稀疏问题。
    
    - 无法表达语义相似性：在数学上，独热编码中“猫”和“狗”的距离，与“猫”和“汽车”的距离是完全一样的。这显然不符合常理，因为猫和狗的语义关系更近。

- `nn.Embedding` 完美地解决了这两个问题。它将每个单词映射到一个维度低得多（例如100维，而不是10,000维）的密集向量。在训练过程中，模型会学习将意思相近的词放在向量空间中相近的位置。

In [ ]:
nn.Embedding(num_embeddings,
             embedding_dim, 
             padding_idx=None, 
             max_norm=None, 
             norm_type=2.0, 
             scale_grad_by_freq=False, 
             sparse=False)


- `num_embeddings` → (整数): 嵌入字典的大小，也就是你的词汇表中唯一词语的总数。例如，如果你的词汇表有1000个不同的单词，那么这个值就应该是1000。。

- `embedding_dim` → (整数): 每个嵌入向量的维度。这是一个需要你手动设置的超参数，常见的值有 50, 100, 256, 300 等。它决定了词向量所在空间的维度。

- `padding_idx` →  (整数, 可选): 填充索引。如果设置了这个参数，那么在训练过程中，对应这个索引的嵌入向量将不会被更新。这在处理可变长度的句子时非常有用，我们可以用一个特殊的索引（比如0）来填充较短的句子，并通过设置 `padding_idx=0` 告诉模型忽略这些填充标记。

- `max_norm` → 每次更新后约束 `embedding` 向量的最大范数。

- `sparse` → 如果为 `True`，梯度计算会用稀疏更新，适合大词表。

---
### 输入与输出的形状 📐
- 输入: 一个 `torch.LongTensor` 类型的张量，里面包含了需要查找的整数索引。形状可以是 `(N)` 表示一个序列，或者 `(N, L)` 表示一个批次 `(batch)` 中包含 N 个长度为 L 的序列。

- 输出: 查找到的对应嵌入向量。`nn.Embedding` 会在输入张量的最后追加一个新的维度，大小为 `embedding_dim`。

如果输入形状是 (N, L)，则输出形状会是 (N, L, `embedding_dim`)。

In [ ]:
import torch
import torch.nn as nn

# --- 参数设置 ---
# 假设我们的词汇表大小为 1000
vocab_size = 1000
# 我们希望用一个 50 维的向量来表示每个词
embedding_dimension = 50
# 批次大小为 4 (4个句子)
batch_size = 4
# 每个句子的长度为 10
sequence_length = 10

# --- 创建 Embedding 层 ---
embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dimension)

# --- 创建模拟输入数据 ---
# 一个批次的 4 个句子，每个句子有 10 个词。
# 张量中的值是每个词在词汇表中的整数索引。
# 注意：索引的值必须小于 vocab_size (1000)。
input_indices = torch.randint(0, vocab_size, (batch_size, sequence_length)) # 形状: (4, 10)

# --- 前向传播 ---
# 将索引输入到 embedding 层
output_vectors = embedding_layer(input_indices) # 形状: (4, 10, 50)

# --- 打印形状 ---
print("输入索引的形状:", input_indices.shape)
print("输出向量的形状:", output_vectors.shape)
print("\n第一个句子的第一个词对应的词向量（示例）:")
print(output_vectors[0, 0, :])

In [ ]:
运行结果:

输入索引的形状: torch.Size([4, 10])
输出向量的形状: torch.Size([4, 10, 50])

第一个句子的第一个词对应的词向量（示例）:
tensor([ 0.3456, -1.2345,  ...,  0.9876, -0.5432], grad_fn=<SliceBackward0>)

正如你所见，nn.Embedding 层接收了我们 (4, 10) 形状的索引张量，并返回了一个 (4, 10, 50) 形状的、由密集向量组成的张量。